# Counter-Strike Match Predicition Model

##### The main goal of this project is to find if the data generated by matches can give as an idea of wich team will win!

# Importing Libraries

In [1]:
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
import requests
from tqdm import tqdm
import re
from datetime import datetime

# Creating connection with DB

### This Data Base was created to scrap Hltv.org web site.

In [2]:
#creating variable with env file location
env_path = os.path.join('/Users/yurivelkis/Downloads/BootCamp/Counter-Strike_Match_Prediction_Model/','secrets.env')
if os.path.exists(env_path):
    load_dotenv(env_path)

#gathering senstive data from file.env
db_server = os.getenv('db_server')
user = os.getenv('user')
db_port = os.getenv('db_port')
password = os.getenv('password')
ip = os.getenv('ip')
db_name = os.getenv('db_name')

#Engine to connect the DB
engine = create_engine(f'mysql+{db_server}://{user}:{password}@{ip}:{db_port}/{db_name}?charset=utf8')

## Function to try connection with DB
def connection(x):
    try:
        conn = x.connect()
        return conn
    except:
        print('Conncection Fail')

# Importing Data From Database 

In [15]:
# Function to grab data and generate DF
def extract_data_from_db(engine):
    conn = connection(engine)
    sql = " SELECT * FROM matches_info "
    matches_scrapped2 = pd.read_sql(sql, con=conn)
    df = pd.DataFrame(matches_scrapped2)
    return df

df = extract_data_from_db(engine)

# Meta Data

## Before starting lets make clear what each database column means.

In [27]:
metadata= {
    'date' : ['Date when the match happened'],
    'match_id' : ['Unique ID to identify match'],
    'team_1' : ['Team one name'],
    'team_2' : ['Team two name]'],
    'best_of' : ['kind of the match, example best_of 3 '],
}

pd.DataFrame(metadata).T

,0
date,Date when the match happened
match_id,Unique ID to identify match
team_1,Team one name
team_2,Team two name]
best_of,"kind of the match, example best_of 3"


# Data Transformation

In [16]:
# 

,date,match_id,team_1,team_2,best_of,map,score_t1,ct_side_t1,t_side_t1,score_t2,...,team_1_clutches,team_2_clutches,eco_t1,eco_t2,semi_eco_t1,semi_eco_t2,semi_buy_t1,semi_buy_t2,full_buy_t1,full_buy_t2
0,2022-09-30 19:15,2358331,G2,Natus Vincere,bo3,Dust2,13,9,4,16,...,0,3,2,1,3,3,4,2,18,21
1,2022-09-30 15:30,2358332,Vitality,Outsiders,bo3,Mirage,16,10,6,13,...,2,0,1,2,3,1,1,2,22,22
2,2022-09-29 19:00,2358329,Cloud9,FaZe,bo3,Inferno,13,6,7,16,...,2,1,1,1,3,1,2,4,21,21
3,2022-09-29 15:45,2358330,MOUZ,Liquid,bo3,Inferno,10,4,6,16,...,1,2,2,0,1,1,4,3,17,20
4,2022-09-28 19:50,2358327,Heroic,Natus Vincere,bo3,Mirage,14,9,5,16,...,1,4,3,2,0,3,5,4,20,19
